In [1]:
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, Input, Flatten

Using TensorFlow backend.


In [2]:
import pandas as pd

imdb_path = "data/imdb/imdb.csv"
data = pd.read_csv(imdb_path)

In [4]:
data.columns

Index(['review', 'sentiment'], dtype='object')

In [9]:
texts = list(data["review"])
labels = list(data["sentiment"])

In [10]:
train_x = texts[:40000]
train_y = labels[:40000]
test_x = texts[40000:]
test_y = labels[40000:]

In [11]:
for idx,row in enumerate(train_y):
    if row == "positive":
        train_y[idx] = 1
    else:
        train_y[idx] = 0
train_y = np.array(train_y)
for idx,row in enumerate(test_y):
    if row == "positive":
        test_y[idx] = 1
    else:
        test_y[idx] = 0
test_y = np.array(test_y)

In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle

Using TensorFlow backend.


In [12]:
input_size = 200
max_len = 100
max_words = 20000

tokenizer = Tokenizer(num_words=max_words+1, oov_token="UNK")
tokenizer.fit_on_texts(train_x)
with open("model/keras_tokenizer_imdb.pickle", "wb") as f:
    pickle.dump(tokenizer, f, pickle.HIGHEST_PROTOCOL)

In [13]:
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= max_words+1}

In [14]:
len(tokenizer.word_index)

20001

In [15]:
train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

KeyboardInterrupt: 

In [ ]:
train_x = pad_sequences(train_x, maxlen=max_len, padding="post", truncating="post")
test_x = pad_sequences(test_x, maxlen=max_len, padding="post", truncating="post")

In [5]:
model = Sequential()
model.add(Embedding(input_dim=max_words+1, output_dim=input_size))
model.add(LSTM(64))
model.add(Dense(1, activation="sigmoid"))

W0817 20:27:57.577946 20428 deprecation_wrapper.py:119] From C:\Users\jeon\AppData\Local\Programs\Python\Python36-32\Lib\conda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0817 20:27:57.651991 20428 deprecation_wrapper.py:119] From C:\Users\jeon\AppData\Local\Programs\Python\Python36-32\Lib\conda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0817 20:27:57.662935 20428 deprecation_wrapper.py:119] From C:\Users\jeon\AppData\Local\Programs\Python\Python36-32\Lib\conda\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 200)         4000200   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                67840     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 4,068,105
Trainable params: 4,068,105
Non-trainable params: 0
_________________________________________________________________


In [187]:
model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["acc"])
history = model.fit(train_x, train_y, epochs=3, batch_size=128, validation_data=(test_x, test_y))

Train on 40000 samples, validate on 10000 samples
Epoch 1/3
40000/40000 [==============================] - 147s 4ms/step - loss: 0.4645 - acc: 0.7667 - val_loss: 0.3667 - val_acc: 0.8361
Epoch 2/3
40000/40000 [==============================] - 129s 3ms/step - loss: 0.2813 - acc: 0.8881 - val_loss: 0.3691 - val_acc: 0.8400
Epoch 3/3
40000/40000 [==============================] - 129s 3ms/step - loss: 0.2035 - acc: 0.9224 - val_loss: 0.4032 - val_acc: 0.8291


In [188]:
model.save_weights("model/keras_imdb_lstm.h5")

In [17]:
test = [
    "so terrible movie"
]
test = tokenizer.texts_to_sequences(test)
test = pad_sequences(test, maxlen=max_len)
model.predict(test)

array([[0.45932412]], dtype=float32)

In [7]:
model.load_weights("model/keras_imdb_lstm.h5")

W0817 20:28:20.183890 20428 deprecation_wrapper.py:119] From C:\Users\jeon\AppData\Local\Programs\Python\Python36-32\Lib\conda\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0817 20:28:20.186894 20428 deprecation_wrapper.py:119] From C:\Users\jeon\AppData\Local\Programs\Python\Python36-32\Lib\conda\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

